In [1]:
import json
import h5py
import numpy as np
import cv2
from pathlib import Path
from typing import Dict, List, Set
from collections import defaultdict
import os
from tqdm import tqdm

class ImageExtractor:
    """从JSONL文件读取信息并从HDF5文件中提取对应图像"""
    
    def __init__(self, 
                 base_data_path: str = "/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/benchmark1_0_compressed",
                 output_base_path: str = "/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/images"):
        """
        初始化提取器
        
        Args:
            base_data_path: HDF5数据的基础路径
            output_base_path: 输出图像的基础路径
        """
        self.base_data_path = Path(base_data_path)
        self.output_base_path = Path(output_base_path)
        self.errors = []  # 记录错误
        
    def parse_id(self, id_string: str) -> Dict[str, str]:
        """
        解析ID字符串
        
        Args:
            id_string: 格式如 "h5_franka_3rgb/place_in_block_in_plate_1/1014_163457"
            
        Returns:
            包含解析结果的字典
        """
        parts = id_string.split('/')
        if len(parts) < 3:
            raise ValueError(f"无效的ID格式: {id_string}")
            
        return {
            'embodiment': parts[0],  # h5_franka_3rgb
            'task_name': parts[1],    # place_in_block_in_plate_1
            'episode_id': parts[2]     # 1014_163457
        }
    
    def construct_h5_path(self, id_info: Dict[str, str]) -> Path:
        """
        根据ID信息构建HDF5文件路径
        
        Args:
            id_info: 包含embodiment, task_name, episode_id的字典
            
        Returns:
            HDF5文件的完整路径
        """
        h5_dir = self.base_data_path / id_info['embodiment'] / id_info['task_name'] / \
                 'success_episodes' / 'train' / id_info['episode_id'] / 'data'
        
        # 检查目录是否存在
        if not h5_dir.exists():
            raise FileNotFoundError(f"目录不存在: {h5_dir}")
        
        # 查找HDF5文件（通常名为trajectory.hdf5）
        h5_files = list(h5_dir.glob('*.hdf5'))
        if not h5_files:
            raise FileNotFoundError(f"未找到HDF5文件在: {h5_dir}")
        
        # 优先选择trajectory.hdf5
        for h5_file in h5_files:
            if h5_file.name == 'trajectory.hdf5':
                return h5_file
        
        return h5_files[0]  # 如果没有trajectory.hdf5，返回第一个HDF5文件
    
    def decode_image(self, compressed_image: np.ndarray) -> np.ndarray:
        """
        解码压缩的图像数据
        
        Args:
            compressed_image: 压缩的图像数据
            
        Returns:
            解码后的RGB图像
        """
        if compressed_image is None:
            return None
            
        # 如果是单个图像
        if len(compressed_image.shape) == 1:
            rgb = cv2.imdecode(compressed_image, cv2.IMREAD_COLOR)
            if rgb is None:
                # 尝试直接解析为数组
                rgb = np.frombuffer(compressed_image, dtype=np.uint8)
                # 根据大小判断图像尺寸
                if rgb.size == 2764800:
                    rgb = rgb.reshape(720, 1280, 3)
                elif rgb.size == 921600:
                    rgb = rgb.reshape(480, 640, 3)
            return rgb
        else:
            # 如果是批量图像，返回None（需要特殊处理）
            return None
    
    def extract_specific_frames(self, h5_path: Path, frame_names: List[str], 
                               camera_name: str = 'camera_top') -> Dict[str, np.ndarray]:
        """
        从HDF5文件中提取特定帧
        
        Args:
            h5_path: HDF5文件路径
            frame_names: 要提取的帧名称列表（如 ["camera_top_0000.jpg", "camera_top_0281.jpg"]）
            camera_name: 相机名称
            
        Returns:
            帧名称到图像数组的映射
        """
        images = {}
        
        with h5py.File(h5_path, 'r') as f:
            # 检查是否压缩
            is_compress = f.attrs.get('compress', True)
            
            # 获取RGB图像数据
            if 'observations' in f and 'rgb_images' in f['observations']:
                if camera_name in f['observations']['rgb_images']:
                    rgb_data = f['observations']['rgb_images'][camera_name]
                    
                    # 解析帧索引
                    for frame_name in frame_names:
                        # 从文件名提取帧索引（如 camera_top_0000.jpg -> 0）
                        try:
                            # 提取数字部分
                            frame_idx_str = frame_name.split('_')[-1].split('.')[0]
                            frame_idx = int(frame_idx_str)
                            
                            if frame_idx < len(rgb_data):
                                if is_compress:
                                    # 解码压缩图像
                                    compressed_img = rgb_data[frame_idx]
                                    img = self.decode_image(compressed_img)
                                    if img is not None:
                                        # 转换BGR到RGB
                                        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                                        images[frame_name] = img
                                else:
                                    # 直接读取未压缩图像
                                    images[frame_name] = rgb_data[frame_idx]
                        except (ValueError, IndexError):
                            continue
            
        return images
    
    def save_images(self, images: Dict[str, np.ndarray], output_dir: Path):
        """
        保存图像到指定目录
        
        Args:
            images: 帧名称到图像数组的映射
            output_dir: 输出目录
        """
        output_dir.mkdir(parents=True, exist_ok=True)
        
        for frame_name, img in images.items():
            output_path = output_dir / frame_name
            # 转换RGB回BGR用于OpenCV保存
            img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            cv2.imwrite(str(output_path), img_bgr)
    
    def process_jsonl(self, jsonl_path: str, limit: int = None):
        """
        处理JSONL文件并提取所有图像
        
        Args:
            jsonl_path: JSONL文件路径
            limit: 限制处理的行数（用于测试）
        """
        # 重置错误列表
        self.errors = []
        
        # 收集所有需要提取的数据
        extraction_tasks = defaultdict(lambda: defaultdict(set))
        
        print(f"📖 读取JSONL文件: {jsonl_path}")
        
        # 读取JSONL文件
        with open(jsonl_path, 'r') as f:
            lines = f.readlines()
            if limit:
                lines = lines[:limit]
                
            for line_num, line in enumerate(lines, 1):
                try:
                    data = json.loads(line.strip())
                    id_string = data['id']
                    
                    # 收集所有需要的帧
                    all_frames = set()
                    
                    # visual_demo中的帧
                    if 'visual_demo' in data:
                        all_frames.update(data['visual_demo'])
                    
                    # stage_to_estimate中的帧
                    if 'stage_to_estimate' in data:
                        all_frames.update(data['stage_to_estimate'])
                    
                    # 按ID和相机分组
                    extraction_tasks[id_string]['frames'].update(all_frames)
                    
                except json.JSONDecodeError:
                    self.errors.append(f"第 {line_num} 行: JSON解析错误")
                except Exception as e:
                    self.errors.append(f"第 {line_num} 行: {str(e)}")
        
        print(f"📊 发现 {len(extraction_tasks)} 个唯一ID需要处理\n")
        
        # 统计成功和失败
        success_count = 0
        failed_count = 0
        
        # 使用tqdm处理每个唯一的ID
        with tqdm(total=len(extraction_tasks), desc="提取图像", unit="ID") as pbar:
            for id_string, task_info in extraction_tasks.items():
                try:
                    # 解析ID
                    id_info = self.parse_id(id_string)
                    
                    # 构建HDF5路径
                    h5_path = self.construct_h5_path(id_info)
                    
                    # 提取所有帧
                    frames_to_extract = list(task_info['frames'])
                    
                    # 从HDF5提取图像
                    images = self.extract_specific_frames(h5_path, frames_to_extract)
                    
                    if images:
                        # 构建输出路径
                        output_dir = self.output_base_path / id_info['embodiment'] / \
                                    id_info['task_name'] / id_info['episode_id']
                        
                        # 保存图像
                        self.save_images(images, output_dir)
                        success_count += 1
                        
                        # 更新进度条描述
                        pbar.set_postfix({
                            '成功': success_count, 
                            '失败': failed_count,
                            '当前': f"{id_info['task_name'][:20]}..."
                        })
                    else:
                        failed_count += 1
                        self.errors.append(f"ID {id_string}: 未能提取任何图像")
                        
                except Exception as e:
                    failed_count += 1
                    self.errors.append(f"ID {id_string}: {str(e)}")
                    pbar.set_postfix({'成功': success_count, '失败': failed_count})
                
                pbar.update(1)
        
        # 输出最终统计
        print(f"\n✅ 处理完成！")
        print(f"   成功: {success_count} 个ID")
        print(f"   失败: {failed_count} 个ID")
        
        # 如果有错误，询问是否查看
        if self.errors:
            print(f"\n⚠️ 发现 {len(self.errors)} 个错误")
            response = input("是否查看错误详情？(y/n): ")
            if response.lower() == 'y':
                print("\n错误列表:")
                for i, error in enumerate(self.errors, 1):
                    print(f"  {i}. {error}")
    
    def diagnose_path(self, id_string: str):
        """
        诊断路径问题，帮助调试
        
        Args:
            id_string: ID字符串
        """
        print(f"\n🔍 诊断ID: {id_string}")
        
        try:
            id_info = self.parse_id(id_string)
            print(f"  解析结果:")
            print(f"    - Embodiment: {id_info['embodiment']}")
            print(f"    - Task: {id_info['task_name']}")
            print(f"    - Episode: {id_info['episode_id']}")
            
            # 检查embodiment目录
            embodiment_path = self.base_data_path / id_info['embodiment']
            print(f"\n  检查embodiment目录: {embodiment_path}")
            if embodiment_path.exists():
                print(f"    ✓ 目录存在")
                
                # 列出所有任务
                task_dirs = sorted([d.name for d in embodiment_path.iterdir() if d.is_dir()])
                print(f"    📁 找到 {len(task_dirs)} 个任务目录")
                
                # 查找相似的任务名
                from difflib import get_close_matches
                similar = get_close_matches(id_info['task_name'], task_dirs, n=3, cutoff=0.6)
                if similar:
                    print(f"    💡 相似的任务名: {similar}")
                
                # 检查确切的任务目录
                task_path = embodiment_path / id_info['task_name']
                if task_path.exists():
                    print(f"\n  ✓ 任务目录存在: {task_path}")
                    
                    # 检查episode路径
                    episode_path = task_path / 'success_episodes' / 'train' / id_info['episode_id']
                    if episode_path.exists():
                        print(f"  ✓ Episode目录存在: {episode_path}")
                        
                        # 检查data目录
                        data_path = episode_path / 'data'
                        if data_path.exists():
                            print(f"  ✓ Data目录存在: {data_path}")
                            
                            # 列出所有文件
                            files = list(data_path.iterdir())
                            print(f"  📁 目录内容:")
                            for f in files:
                                print(f"      - {f.name} ({f.stat().st_size / 1024 / 1024:.2f} MB)")
                            
                            # 查找HDF5文件
                            h5_files = list(data_path.glob('*.hdf5'))
                            if h5_files:
                                print(f"  ✅ 找到HDF5文件: {h5_files[0].name}")
                            else:
                                print(f"  ❌ 未找到HDF5文件")
                        else:
                            print(f"  ❌ Data目录不存在")
                    else:
                        print(f"  ❌ Episode目录不存在")
                        # 列出可用的episodes
                        train_path = task_path / 'success_episodes' / 'train'
                        if train_path.exists():
                            episodes = sorted([d.name for d in train_path.iterdir() if d.is_dir()])[:5]
                            print(f"  💡 可用的episodes示例: {episodes}")
                else:
                    print(f"  ❌ 任务目录不存在: {id_info['task_name']}")
                    print(f"  💡 请检查任务名是否正确")
            else:
                print(f"    ❌ Embodiment目录不存在")
                
        except Exception as e:
            print(f"  ❌ 诊断失败: {e}")
    
    def list_available_tasks(self, embodiment: str = "h5_franka_3rgb"):
        """
        列出指定embodiment下的所有可用任务
        
        Args:
            embodiment: embodiment名称
        """
        embodiment_path = self.base_data_path / embodiment
        
        if not embodiment_path.exists():
            print(f"❌ Embodiment目录不存在: {embodiment_path}")
            return []
        
        tasks = []
        for task_dir in sorted(embodiment_path.iterdir()):
            if task_dir.is_dir():
                # 检查是否有success_episodes/train目录
                train_path = task_dir / 'success_episodes' / 'train'
                if train_path.exists():
                    episodes = list(train_path.iterdir())
                    if episodes:
                        tasks.append({
                            'name': task_dir.name,
                            'episodes': len(episodes)
                        })
        
        print(f"\n📋 {embodiment} 下的可用任务:")
        print(f"{'任务名称':<50} {'Episodes数量':>15}")
        print("-" * 65)
        for task in tasks:
            print(f"{task['name']:<50} {task['episodes']:>15}")
        
        return tasks


# ========== 使用示例 ==========

def main():
    # 创建提取器
    extractor = ImageExtractor(
        base_data_path="/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/benchmark1_0_compressed",
        output_base_path="/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/images"
    )
    
    # 处理JSONL文件
    jsonl_file = "/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/annotation/h5_tienkung_xsens_converted.jsonl"
    extractor.process_jsonl(jsonl_file)
    
    # 如果需要诊断特定ID的问题，可以使用：
    # extractor.diagnose_path("h5_franka_3rgb/blue_cub_on_pink/1015_172846")
    
    # 列出可用任务：
    # extractor.list_available_tasks("h5_franka_3rgb")


if __name__ == "__main__":
    main()
# h5_ur_1rgb/triangle_bread_on_table/1015_112426 missed

📖 读取JSONL文件: /home/runsheng/personal_3/qiancx/Sources/datasets/robomind/annotation/h5_tienkung_xsens_converted.jsonl
📊 发现 1046 个唯一ID需要处理



提取图像: 100%|██████████| 1046/1046 [01:21<00:00, 12.82ID/s, 成功=957, 失败=89, 当前=wipe_panel...]            


✅ 处理完成！
   成功: 957 个ID
   失败: 89 个ID

⚠️ 发现 89 个错误



错误列表:
  1. ID h5_tienkung_xsens_1rgb/battery_insertion_with_pullout/2024-09-26-15-08-05: 目录不存在: /home/runsheng/personal_3/qiancx/Sources/datasets/robomind/benchmark1_0_compressed/h5_tienkung_xsens_1rgb/battery_insertion_with_pullout/success_episodes/train/2024-09-26-15-08-05/data
  2. ID h5_tienkung_xsens_1rgb/battery_insertion_with_pullout/2024-09-26-10-25-07: 目录不存在: /home/runsheng/personal_3/qiancx/Sources/datasets/robomind/benchmark1_0_compressed/h5_tienkung_xsens_1rgb/battery_insertion_with_pullout/success_episodes/train/2024-09-26-10-25-07/data
  3. ID h5_tienkung_xsens_1rgb/brick_piled_then_press_thrice/2024-10-18-11-26-48: 目录不存在: /home/runsheng/personal_3/qiancx/Sources/datasets/robomind/benchmark1_0_compressed/h5_tienkung_xsens_1rgb/brick_piled_then_press_thrice/success_episodes/train/2024-10-18-11-26-48/data
  4. ID h5_tienkung_xsens_1rgb/gear_place/2024-10-09-14-25-11: 目录不存在: /home/runsheng/personal_3/qiancx/Sources/datasets/robomind/benchmark1_0_compressed/h5_tienkung_xsens

In [2]:
import json
from pathlib import Path
from typing import Dict, List
from tqdm import tqdm

class JSONLCleaner:
    """检查JSONL文件中的样本，移除图像不存在的样本"""
    
    def __init__(self, images_base_path: str):
        """
        初始化清理器
        
        Args:
            images_base_path: 图像文件的基础路径
        """
        self.images_base_path = Path(images_base_path)
        self.missing_samples = []  # 记录缺失的样本
        self.missing_images = []   # 记录缺失的图像
        
    def parse_id(self, id_string: str) -> Dict[str, str]:
        """
        解析ID字符串
        
        Args:
            id_string: 格式如 "h5_franka_3rgb/place_in_block_in_plate_1/1014_163457"
            
        Returns:
            包含解析结果的字典
        """
        parts = id_string.split('/')
        if len(parts) < 3:
            raise ValueError(f"无效的ID格式: {id_string}")
            
        return {
            'embodiment': parts[0],
            'task_name': parts[1],
            'episode_id': parts[2]
        }
    
    def check_image_exists(self, id_string: str, image_names: List[str]) -> tuple[bool, List[str]]:
        """
        检查指定样本的所有图像是否存在
        
        Args:
            id_string: ID字符串
            image_names: 图像文件名列表
            
        Returns:
            (是否所有图像都存在, 缺失的图像列表)
        """
        try:
            # 解析ID
            id_info = self.parse_id(id_string)
            
            # 构建图像目录路径
            image_dir = self.images_base_path / id_info['embodiment'] / \
                       id_info['task_name'] / id_info['episode_id']
            
            # 检查目录是否存在
            if not image_dir.exists():
                return False, image_names
            
            # 检查每个图像文件是否存在
            missing = []
            for image_name in image_names:
                image_path = image_dir / image_name
                if not image_path.exists():
                    missing.append(image_name)
            
            # 如果有任何图像缺失，返回False
            return len(missing) == 0, missing
            
        except Exception as e:
            # 如果解析失败或其他错误，认为图像不存在
            return False, image_names
    
    def clean_jsonl(self, input_jsonl: str, output_jsonl: str):
        """
        清理JSONL文件，移除图像不存在的样本
        
        Args:
            input_jsonl: 输入JSONL文件路径
            output_jsonl: 输出JSONL文件路径
        """
        # 重置统计
        self.missing_samples = []
        self.missing_images = []
        
        print(f"📖 读取JSONL文件: {input_jsonl}")
        
        # 读取所有行
        with open(input_jsonl, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        print(f"📊 总共 {len(lines)} 个样本")
        print(f"🔍 开始检查图像是否存在...\n")
        
        valid_lines = []
        removed_count = 0
        
        # 使用tqdm显示进度
        with tqdm(total=len(lines), desc="检查样本", unit="样本") as pbar:
            for line_num, line in enumerate(lines, 1):
                try:
                    data = json.loads(line.strip())
                    id_string = data['id']
                    
                    # 收集所有需要检查的图像
                    all_images = set()
                    
                    if 'visual_demo' in data:
                        all_images.update(data['visual_demo'])
                    
                    if 'stage_to_estimate' in data:
                        all_images.update(data['stage_to_estimate'])
                    
                    # 检查图像是否存在
                    all_exist, missing = self.check_image_exists(id_string, list(all_images))
                    
                    if all_exist:
                        # 图像都存在，保留这一行
                        valid_lines.append(line)
                    else:
                        # 有图像不存在，记录并移除
                        removed_count += 1
                        self.missing_samples.append({
                            'line_num': line_num,
                            'id': id_string,
                            'missing_images': missing
                        })
                        
                        for img in missing:
                            self.missing_images.append({
                                'id': id_string,
                                'image': img
                            })
                    
                    # 更新进度条
                    pbar.set_postfix({
                        '保留': len(valid_lines),
                        '移除': removed_count
                    })
                    
                except json.JSONDecodeError:
                    print(f"\n⚠️ 第 {line_num} 行: JSON解析错误，已跳过")
                    removed_count += 1
                except Exception as e:
                    print(f"\n⚠️ 第 {line_num} 行: {str(e)}，已跳过")
                    removed_count += 1
                
                pbar.update(1)
        
        # 写入新的JSONL文件
        print(f"\n💾 写入清理后的JSONL文件: {output_jsonl}")
        output_path = Path(output_jsonl)
        output_path.parent.mkdir(parents=True, exist_ok=True)
        
        with open(output_jsonl, 'w', encoding='utf-8') as f:
            for line in valid_lines:
                f.write(line)
        
        # 输出统计信息
        print(f"\n✅ 清理完成！")
        print(f"   原始样本数: {len(lines)}")
        print(f"   保留样本数: {len(valid_lines)}")
        print(f"   移除样本数: {removed_count}")
        print(f"   保留率: {len(valid_lines)/len(lines)*100:.2f}%")
        
        # 询问是否查看缺失详情
        if self.missing_samples:
            print(f"\n⚠️ 发现 {len(self.missing_samples)} 个样本的图像缺失")
            response = input("是否查看缺失详情？(y/n): ")
            if response.lower() == 'y':
                self.show_missing_details()
    
    def show_missing_details(self):
        """显示缺失图像的详细信息"""
        print("\n" + "="*80)
        print("缺失样本详情")
        print("="*80)
        
        for i, sample in enumerate(self.missing_samples[:50], 1):  # 最多显示50个
            print(f"\n{i}. 行号: {sample['line_num']}")
            print(f"   ID: {sample['id']}")
            print(f"   缺失图像 ({len(sample['missing_images'])} 个):")
            for img in sample['missing_images'][:10]:  # 每个样本最多显示10个图像
                print(f"      - {img}")
            if len(sample['missing_images']) > 10:
                print(f"      ... 还有 {len(sample['missing_images']) - 10} 个图像")
        
        if len(self.missing_samples) > 50:
            print(f"\n... 还有 {len(self.missing_samples) - 50} 个缺失样本未显示")
        
        print("\n" + "="*80)
    
    def generate_missing_report(self, report_path: str):
        """
        生成缺失图像的详细报告
        
        Args:
            report_path: 报告文件路径（JSON格式）
        """
        report = {
            'total_missing_samples': len(self.missing_samples),
            'total_missing_images': len(self.missing_images),
            'missing_samples': self.missing_samples
        }
        
        with open(report_path, 'w', encoding='utf-8') as f:
            json.dump(report, f, indent=2, ensure_ascii=False)
        
        print(f"📄 缺失报告已生成: {report_path}")


# ========== 使用示例 ==========

def main():
    # ===== 路径参数定义 =====
    IMAGES_BASE_PATH = "/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/images"
    INPUT_JSONL = "/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/annotation/h5_tienkung_xsens_converted.jsonl"
    OUTPUT_JSONL = "/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/annotation/h5_tienkung_xsens_converted_cleaned.jsonl"
    REPORT_PATH = "/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/annotation/missing_report.json"
    
    # ===== 创建清理器并执行 =====
    cleaner = JSONLCleaner(images_base_path=IMAGES_BASE_PATH)
    
    # 清理JSONL文件
    cleaner.clean_jsonl(
        input_jsonl=INPUT_JSONL,
        output_jsonl=OUTPUT_JSONL
    )
    
    # 如果需要生成详细报告，取消下面的注释：
    # cleaner.generate_missing_report(REPORT_PATH)


if __name__ == "__main__":
    main()

📖 读取JSONL文件: /home/runsheng/personal_3/qiancx/Sources/datasets/robomind/annotation/h5_tienkung_xsens_converted.jsonl
📊 总共 17811 个样本
🔍 开始检查图像是否存在...



检查样本: 100%|██████████| 17811/17811 [00:13<00:00, 1288.10样本/s, 保留=16359, 移除=1452]



💾 写入清理后的JSONL文件: /home/runsheng/personal_3/qiancx/Sources/datasets/robomind/annotation/h5_tienkung_xsens_converted_cleaned.jsonl

✅ 清理完成！
   原始样本数: 17811
   保留样本数: 16359
   移除样本数: 1452
   保留率: 91.85%

⚠️ 发现 1452 个样本的图像缺失

缺失样本详情

1. 行号: 3379
   ID: h5_tienkung_xsens_1rgb/battery_insertion_with_pullout/2024-09-26-15-08-05
   缺失图像 (8 个):
      - camera_top_0128.jpg
      - camera_top_0210.jpg
      - camera_top_0105.jpg
      - camera_top_0000.jpg
      - camera_top_0337.jpg
      - camera_top_0035.jpg
      - camera_top_0011.jpg
      - camera_top_0163.jpg

2. 行号: 3380
   ID: h5_tienkung_xsens_1rgb/battery_insertion_with_pullout/2024-09-26-15-08-05
   缺失图像 (8 个):
      - camera_top_0128.jpg
      - camera_top_0210.jpg
      - camera_top_0105.jpg
      - camera_top_0000.jpg
      - camera_top_0337.jpg
      - camera_top_0035.jpg
      - camera_top_0066.jpg
      - camera_top_0163.jpg

3. 行号: 3381
   ID: h5_tienkung_xsens_1rgb/battery_insertion_with_pullout/2024-09-26-15-08-05
   缺失图像 (8

In [4]:
import json
from pathlib import Path
from typing import Dict, List
from tqdm import tqdm

class JSONLValidator:
    """验证JSONL文件中的所有样本是否都能找到对应的图像，并在命令行显示统计结果"""
    
    def __init__(self, images_base_path: str):
        """
        初始化验证器
        
        Args:
            images_base_path: 图像文件的基础路径
        """
        self.images_base_path = Path(images_base_path)
        self.stats = {
            'total_samples': 0,
            'valid_samples': 0,
            'invalid_samples': 0,
            'total_images_checked': 0,
            'missing_images_count': 0
        }
        
    def parse_id(self, id_string: str) -> Dict[str, str]:
        """
        解析ID字符串
        
        Args:
            id_string: 格式如 "h5_franka_3rgb/place_in_block_in_plate_1/1014_163457"
            
        Returns:
            包含解析结果的字典
        """
        parts = id_string.split('/')
        if len(parts) < 3:
            raise ValueError(f"无效的ID格式: {id_string}")
            
        return {
            'embodiment': parts[0],
            'task_name': parts[1],
            'episode_id': parts[2]
        }
    
    def check_images(self, id_string: str, image_names: List[str]) -> Dict:
        """
        检查指定样本的所有图像是否存在
        
        Args:
            id_string: ID字符串
            image_names: 图像文件名列表
            
        Returns:
            验证结果字典
        """
        result = {
            'id': id_string,
            'total_images': len(image_names),
            'exists': [],
            'missing': [],
            'directory_exists': False,
            'all_images_exist': False
        }
        
        try:
            # 解析ID
            id_info = self.parse_id(id_string)
            
            # 构建图像目录路径
            image_dir = self.images_base_path / id_info['embodiment'] / \
                       id_info['task_name'] / id_info['episode_id']
            
            # 检查目录是否存在
            result['directory_exists'] = image_dir.exists()
            result['directory_path'] = str(image_dir)
            
            if not image_dir.exists():
                result['missing'] = image_names
                return result
            
            # 检查每个图像文件是否存在
            for image_name in image_names:
                image_path = image_dir / image_name
                if image_path.exists():
                    result['exists'].append(image_name)
                else:
                    result['missing'].append(image_name)
            
            # 判断是否所有图像都存在
            result['all_images_exist'] = len(result['missing']) == 0
            
        except Exception as e:
            result['error'] = str(e)
            result['missing'] = image_names
        
        return result
    
    def validate_jsonl(self, jsonl_path: str):
        """
        验证JSONL文件中的所有样本
        
        Args:
            jsonl_path: JSONL文件路径
        """
        # 重置统计
        self.stats = {
            'total_samples': 0,
            'valid_samples': 0,
            'invalid_samples': 0,
            'total_images_checked': 0,
            'missing_images_count': 0,
            'missing_directories': 0
        }
        
        print(f"📖 读取JSONL文件: {jsonl_path}")
        
        # 读取所有行
        with open(jsonl_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        self.stats['total_samples'] = len(lines)
        print(f"📊 总共 {len(lines)} 个样本")
        print(f"🔍 开始验证图像...\n")
        
        # 使用tqdm显示进度
        with tqdm(total=len(lines), desc="验证样本", unit="样本") as pbar:
            for line_num, line in enumerate(lines, 1):
                try:
                    data = json.loads(line.strip())
                    id_string = data['id']
                    
                    # 收集所有需要检查的图像
                    all_images = set()
                    
                    if 'visual_demo' in data:
                        all_images.update(data['visual_demo'])
                    
                    if 'stage_to_estimate' in data:
                        all_images.update(data['stage_to_estimate'])
                    
                    # 验证图像
                    result = self.check_images(id_string, list(all_images))
                    
                    # 更新统计
                    self.stats['total_images_checked'] += result['total_images']
                    self.stats['missing_images_count'] += len(result['missing'])
                    
                    if result['all_images_exist']:
                        self.stats['valid_samples'] += 1
                    else:
                        self.stats['invalid_samples'] += 1
                        if not result['directory_exists']:
                            self.stats['missing_directories'] += 1
                    
                    # 更新进度条
                    pbar.set_postfix({
                        '通过': self.stats['valid_samples'],
                        '失败': self.stats['invalid_samples']
                    })
                    
                except json.JSONDecodeError:
                    print(f"\n⚠️ 第 {line_num} 行: JSON解析错误")
                    self.stats['invalid_samples'] += 1
                except Exception as e:
                    print(f"\n⚠️ 第 {line_num} 行: {str(e)}")
                    self.stats['invalid_samples'] += 1
                
                pbar.update(1)
        
        # 输出验证结果
        self.print_summary()
    
    def print_summary(self):
        """打印验证摘要"""
        print("\n" + "="*80)
        print("验证结果摘要")
        print("="*80)
        
        total = self.stats['total_samples']
        valid = self.stats['valid_samples']
        invalid = self.stats['invalid_samples']
        
        print(f"\n📊 样本统计:")
        print(f"   总样本数: {total}")
        print(f"   ✅ 通过验证: {valid} ({valid/total*100:.2f}%)")
        print(f"   ❌ 验证失败: {invalid} ({invalid/total*100:.2f}%)")
        
        print(f"\n🖼️ 图像统计:")
        print(f"   总图像数: {self.stats['total_images_checked']}")
        print(f"   ❌ 缺失图像: {self.stats['missing_images_count']}")
        print(f"   ✅ 存在图像: {self.stats['total_images_checked'] - self.stats['missing_images_count']}")
        
        if self.stats['missing_directories'] > 0:
            print(f"\n⚠️ 目录统计:")
            print(f"   缺失目录: {self.stats['missing_directories']} 个样本的目录不存在")
        
        print("\n" + "="*80)
    



# ========== 使用示例 ==========

def main():
    # ===== 路径参数定义 =====
    IMAGES_BASE_PATH = "/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/images"
    JSONL_FILE = "/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/annotation/h5_tienkung_xsens_converted_cleaned.jsonl"
    
    # ===== 创建验证器并执行 =====
    validator = JSONLValidator(images_base_path=IMAGES_BASE_PATH)
    
    # 验证JSONL文件
    validator.validate_jsonl(jsonl_path=JSONL_FILE)


if __name__ == "__main__":
    main()

📖 读取JSONL文件: /home/runsheng/personal_3/qiancx/Sources/datasets/robomind/annotation/h5_tienkung_xsens_converted_cleaned.jsonl
📊 总共 16359 个样本
🔍 开始验证图像...



验证样本:   1%|          | 167/16359 [00:00<00:10, 1533.36样本/s, 通过=168, 失败=0]

验证样本: 100%|██████████| 16359/16359 [00:12<00:00, 1338.49样本/s, 通过=16359, 失败=0]



验证结果摘要

📊 样本统计:
   总样本数: 16359
   ✅ 通过验证: 16359 (100.00%)
   ❌ 验证失败: 0 (0.00%)

🖼️ 图像统计:
   总图像数: 130819
   ❌ 缺失图像: 0
   ✅ 存在图像: 130819

